In [ ]:
epochs = 10
# Não usamos todo o conjunto de dados para fins de eficiência, mas fique à vontade para aumentar esses números
n_train_items = 640
n_test_items = 640

# Parte X - Treinamento e Avaliação Seguros no MNIST

Ao criar soluções de Aprendizado de Máquina como Serviço (MLaaS), uma empresa pode precisar solicitar acesso a dados de outros parceiros para treinar seu modelo. Na saúde ou nas finanças, o modelo e os dados são extremamente críticos: os parâmetros do modelo são um ativo de negócios, enquanto os dados são dados pessoais que são rigorosamente regulamentados.

Nesse contexto, uma solução possível é criptografar o modelo e os dados e treinar o modelo de aprendizado de máquina sobre os valores criptografados. Isso garante que a empresa não acesse os registros médicos dos pacientes, por exemplo, e que os estabelecimentos de saúde não possam observar o modelo para o qual contribuem. Existem vários esquemas de criptografia que permitem a computação sobre dados criptografados, entre os quais a Computação Multipartidária Segura (SMPC - Secure Multipart Computation), a Criptografia Homomórfica (FHE / SHE) e a Criptografia Funcional (FE). Vamos nos concentrar aqui na Computação Multipartidária (que foi introduzida no Tutorial 5), que consiste no compartilhamento de aditivos privados e depende dos protocolos de criptografia SecureNN e SPDZ.

A configuração exata deste tutorial é a seguinte: considere que você é o servidor e gostaria de treinar seu modelo em alguns dados mantidos por $n$ workers. O segredo do servidor compartilha seu modelo e envia cada compartilhamento a um worker. Os workers também compartilham seus dados em segredo e os trocam entre eles. Na configuração que iremos estudar, existem 2 workers: alice e bob. Após a troca de ações, cada um deles agora possui uma de suas próprias ações, uma ação do outro worker e uma ação do modelo. A computação agora pode começar a treinar o modelo em particular, usando os protocolos criptográficos apropriados. Depois que o modelo é treinado, todos os compartilhamentos podem ser enviados de volta ao servidor para descriptografá-lo. Isso é ilustrado com a figura a seguir:

![SMPC Illustration](https://github.com/OpenMined/PySyft/raw/11c85a121a1a136e354945686622ab3731246084/examples/tutorials/material/smpc_illustration.png)

Para dar um exemplo desse processo, vamos assumir que alice e bob mantêm uma parte do conjunto de dados MNIST e vamos treinar um modelo para executar a classificação de dígitos!

Autor:
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel) · GitHub: [@LaRiffle](https://github.com/LaRiffle)

# 1. Demonstração de treinamento criptografado no MNIST

## Configuração de importações e treinamento

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import time

Esta classe descreve todos os hiperparâmetros para o treinamento. Observe que todos eles são públicos aqui.

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.02
        self.seed = 1
        self.log_interval = 1 # Informações de log em cada lote
        self.precision_fractional = 3

args = Arguments()

_ = torch.manual_seed(args.seed)

Aqui estão as importações do PySyft. Nós nos conectamos a dois workers remotos que são chamados `alice` e `bob` e solicitamos outro worker chamado `crypto_provider`, que fornece todas as primitivas criptográficas que precisamos.

In [ ]:
import syft as sy  # importe a biblioteca PySyft
hook = sy.TorchHook(torch)  # conecte o PyTorch para adicionar funcionalidades extras, como Federated e Encrypted Learning

# funções de simulação
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]
def connect_to_crypto_provider():
    return sy.VirtualWorker(hook, id="crypto_provider")

workers = connect_to_workers(n_workers=2)
crypto_provider = connect_to_crypto_provider()

## Obtendo acesso e compartilhamento dados secretos

Aqui, estamos usando uma função de utilitário que simula o seguinte comportamento: assumimos que o conjunto de dados MNIST é distribuído em partes, cada uma das quais mantida por um de nossos workers. Os workers então dividem seus dados em lotes e compartilham seus dados em segredo. O objeto final retornado é iterável nesses lotes secretos compartilhados, que chamamos de **carregador de dados privados**. Observe que durante o processo o trabalhador local (assim nós) nunca teve acesso aos dados.

Obtemos, como de costume, um conjunto de dados privados de treinamento e teste, e as entradas e os rótulos são compartilhados em segredo.

In [ ]:
def get_private_data_loaders(precision_fractional, workers, crypto_provider):
    
    def one_hot_of(index_tensor):
        """
        Transformar em one hot tensor
        
        Examplo:
            [0, 3, 9]
            =>
            [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]
            
        """
        onehot_tensor = torch.zeros(*index_tensor.shape, 10) # 10 classes para o MNIST
        onehot_tensor = onehot_tensor.scatter(1, index_tensor.view(-1, 1), 1)
        return onehot_tensor
        
    def secret_share(tensor):
        """
        Transformar em precisão fixa and compartilhar secretamente um tensor
        """
        return (
            tensor
            .fix_precision(precision_fractional=precision_fractional)
            .share(*workers, crypto_provider=crypto_provider, requires_grad=True)
        )
    
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True, transform=transformation),
        batch_size=args.batch_size
    )
    
    private_train_loader = [
        (secret_share(data), secret_share(one_hot_of(target)))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True, transform=transformation),
        batch_size=args.test_batch_size
    )
    
    private_test_loader = [
        (secret_share(data), secret_share(target.float()))
        for i, (data, target) in enumerate(test_loader)
        if i < n_test_items / args.test_batch_size
    ]
    
    return private_train_loader, private_test_loader
    
    
private_train_loader, private_test_loader = get_private_data_loaders(
    precision_fractional=args.precision_fractional,
    workers=workers,
    crypto_provider=crypto_provider
)

## Especificação do modelo

Aqui está o modelo que usaremos, é bastante simples, mas [provou ter um desempenho razoavelmente bom no MNIST](https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Funções de treinamento e teste

O treinamento é realizado quase como de costume, a diferença real é que não podemos usar funções de custo como negative log-likelihood (`F.nll_loss` no PyTorch) porque é bastante complicado reproduzir essas funções com o SMPC. Em vez disso, usamos uma função mais simples de erro médio quadrático.

In [ ]:
def train(args, model, private_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(private_train_loader): # <-- agora é um conjunto de dados privado
        start_time = time.time()
        
        optimizer.zero_grad()
        
        output = model(data)
        
        # loss = F.nll_loss(output, target)  <-- não é possível aqui
        batch_size = output.shape[0]
        loss = ((output - target)**2).sum().refresh()/batch_size
        
        loss.backward()
        
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            loss = loss.get().float_precision()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.3f}s'.format(
                epoch, batch_idx * args.batch_size, len(private_train_loader) * args.batch_size,
                100. * batch_idx / len(private_train_loader), loss.item(), time.time() - start_time))
            

A função de teste não muda!

In [ ]:
def test(args, model, private_test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in private_test_loader:
            start_time = time.time()
            
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum()

    correct = correct.get().float_precision()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct.item(), len(private_test_loader)* args.test_batch_size,
        100. * correct.item() / (len(private_test_loader) * args.test_batch_size)))

### Vamos iniciar o treinamento!

Algumas notas sobre o que está acontecendo aqui. Primeiro, compartilhamos secretamente todos os parâmetros do modelo entre nossos funcionários. Segundo, convertemos os hiperparâmetros do otimizador em precisão fixa. Observe que não precisamos compartilhá-los em segredo porque eles são públicos em nosso contexto, mas como os valores compartilhados em segredo vivem em campos finitos, ainda precisamos movê-los em campos finitos usando `.fix_precision`, para executar operações consistentes como a atualização de peso $W \leftarrow W - \alpha * \Delta W$.

In [ ]:
model = Net()
model = model.fix_precision().share(*workers, crypto_provider=crypto_provider, requires_grad=True)

optimizer = optim.SGD(model.parameters(), lr=args.lr)
optimizer = optimizer.fix_precision() 

for epoch in range(1, args.epochs + 1):
    train(args, model, private_train_loader, optimizer, epoch)
    test(args, model, private_test_loader)

Aí está você! Você obtém 75% de precisão usando uma pequena fração do conjunto de dados MNIST, usando treinamento 100% criptografado!

# 2. Discussão

Vamos dar uma olhada no poder do treinamento criptografado analisando o que acabamos de fazer.

## 2.1 Tempo de computação

A primeira coisa é obviamente o tempo de execução! Como você certamente notou, é muito mais lento que o treinamento em texto simples. Em particular, uma iteração sobre 1 lote de 64 itens leva 3,2s, enquanto apenas 13ms no PyTorch puro. Embora isso possa parecer um bloqueador, lembre-se de que aqui tudo aconteceu remotamente e no mundo criptografado: nenhum item de dados foi divulgado. Mais especificamente, o tempo para processar um item é de 50 ms, o que não é tão ruim assim. A verdadeira questão é analisar quando o treinamento criptografado é necessário e quando apenas a previsão criptografada é suficiente. 50ms para realizar uma previsão é completamente aceitável em um cenário pronto para produção, por exemplo!

Um dos principais gargalos é o uso de funções de ativação dispendiosas: a ativação da relu com o SMPC é muito dispendiosa porque utiliza comparação privada e o protocolo SecureNN. Como ilustração, se substituirmos relu por uma ativação quadrática, como é feito em vários trabalhos sobre computação criptografada como CryptoNets, passamos de 3,2s para 1,2s.

Como regra geral, a principal idéia a ser retirada é criptografar apenas o necessário, e este tutorial mostra como pode ser simples.

## 2.2 Backpropagation com SMPC

Você pode se perguntar como realizamos atualizações de retropropagação e gradiente, embora estejamos trabalhando com números inteiros em campos finitos. Para isso, desenvolvemos um novo tensor de eixo chamado AutogradTensor. Este tutorial o utilizou intensivamente, embora você talvez não o tenha visto! Vamos verificar isso imprimindo o peso de um modelo:

In [ ]:
model.fc3.bias

E um item dos dados:

In [ ]:
first_batch, input_data = 0, 0
private_train_loader[first_batch][input_data]

Como você observa, o AutogradTensor está lá! Ele vive entre o wrapper do torch e o FixedPrecisionTensor, que indica que os valores agora estão em campos finitos. O objetivo deste AutogradTensor é armazenar o gráfico de computação quando operações são feitas com valores criptografados. Isso é útil porque, ao chamar para trás a backpropagation, esse AutogradTensor substitui todas as funções anteriores que não são compatíveis com a computação criptografada e indica como calcular esses gradientes. Por exemplo, em relação à multiplicação que é feita usando o truque Beaver triples, não queremos diferenciar ainda mais o fato de que diferenciar uma multiplicação deve ser muito fácil: $\partial_b (a \cdot b) = a \cdot \partial b$. Aqui está como descrevemos como calcular esses gradientes, por exemplo:

```python
class MulBackward(GradFunc):
    def __init__(self, self_, other):
        super().__init__(self, self_, other)
        self.self_ = self_
        self.other = other

    def gradient(self, grad):
        grad_self_ = grad * self.other
        grad_other = grad * self.self_ if type(self.self_) == type(self.other) else None
        return (grad_self_, grad_other)
```

Você pode dar uma olhada em `tensors/interpreters/gradients.py` se estiver curioso para ver como implementamos mais gradientes.

Em termos de grafos de computação, significa que uma cópia do grafo permanece local e que o servidor que coordena a passagem para frente também fornece instruções sobre como fazer a passagem para trás. Essa é uma hipótese completamente válida em nosso cenário.

## 2.3 Garantias de segurança


Por fim, vamos dar algumas dicas sobre a segurança que estamos alcançando aqui: os adversários que estamos considerando aqui são **honestos, mas curiosos**: isso significa que um adversário não pode aprender nada sobre os dados executando este protocolo, mas um adversário mal-intencionado ainda pode se desviar do protocolo e, por exemplo, tentar corromper os compartilhamentos para sabotar a computação. A segurança contra adversários mal-intencionados em tais cálculos SMPC, incluindo comparação privada, ainda é um problema em aberto.

Além disso, mesmo que a Computação Multipartidária Segura (SMPC) garanta que os dados de treinamento não foram acessados, muitas ameaças do mundo do texto sem formatação ainda estão presentes aqui. Por exemplo, como você pode fazer uma solicitação ao modelo (no contexto do MLaaS), é possível obter previsões que podem divulgar informações sobre o conjunto de dados de treinamento. Em particular, você não tem proteção contra ataques de associação, um ataque comum a serviços de aprendizado de máquina em que o adversário deseja determinar se um item específico foi usado no conjunto de dados. Além disso, outros ataques, como processos de memorização não intencional (modelos que aprendem recursos específicos sobre um item de dados), inversão ou extração de modelos ainda são possíveis.

Uma solução geral que é eficaz para muitas das ameaças mencionadas acima é adicionar Privacidade Diferencial. Ele pode ser combinado com a Computação Multipartidária Segura e pode fornecer garantias de segurança muito interessantes. No momento, estamos trabalhando em várias implementações e esperamos propor um exemplo que combine as duas em breve!

# Conclusão

Como você viu, o treinamento de um modelo usando SMPC não é complicado do ponto de vista do código, mesmo usando objetos bastante complexos por debaixo dos panos. Com isso em mente, agora você deve analisar seus casos de uso para ver quando a computação criptografada é necessária para treinamento ou avaliação. Se a computação criptografada for muito mais lenta em geral, também poderá ser usada com cuidado para reduzir a sobrecarga geral da computação.

Se você gostou disso e gostaria de participar do movimento de preservação da privacidade, propriedade descentralizada da IA e da cadeia de suprimentos da AI (dados), você pode fazê-lo das seguintes maneiras!

### Dê-nos uma estrela em nosso repo do PySyft no GitHub

A maneira mais fácil de ajudar nossa comunidade é adicionando uma estrela nos nossos repositórios! Isso ajuda a aumentar a conscientização sobre essas ferramentas legais que estamos construindo.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Veja nossos tutoriais no GitHub!

Fizemos tutoriais muito bons para entender melhor como deve ser a Aprendizagem Federada e Preservante de Privacidade e como estamos construindo os tijolos para que isso aconteça.

- [Confira os tutoriais do PySyft](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Junte-se ao Slack!

A melhor maneira de manter-se atualizado sobre os últimos avanços é se juntar à nossa comunidade! Você pode fazer isso preenchendo o formulário em http://slack.openmined.org


### Contribua com o projeto!

A melhor maneira de contribuir para a nossa comunidade é se tornando um contribuidor do código! A qualquer momento, você pode acessar a página de Issues (problemas) do PySyft no GitHub e filtrar por "Projetos". Isso mostrará todas as etiquetas (tags) na parte superior, com uma visão geral de quais projetos você pode participar! Se você não deseja ingressar em um projeto, mas gostaria de codificar um pouco, também pode procurar mais mini-projetos "independentes" pesquisando problemas no GitHub marcados como `good first issue`.

- [Projetos do PySyft](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Etiquetados como Good First Issue](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Doar

Se você não tem tempo para contribuir com nossa base de códigos, mas ainda deseja nos apoiar, também pode se tornar um Apoiador em nosso Open Collective. Todas as doações vão para hospedagem na web e outras despesas da comunidade, como hackathons e meetups!

- [Página do Open Collective do OpenMined](https://opencollective.com/openmined)